In [1]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
import copy

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import itertools
import copy
import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL, DfToSqlMap
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutageDAQ import OutageDAQ, OutageDAQOutg, OutageDAQOtBL, OutageDAQPrBL
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_Box_sns
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSingleSlicer as DFSingleSlicer

In [2]:
    # def __init__(
    #     self,
    #     dataset                , 
    #     run_date               , 
    #     date_0                 ,  
    #     date_1                 , 
    #     run_date_subdir_appndx = None, 
    #     data_evs_sum_vw        = False, 
    #     data_base_dir          = None, 
    #     verbose                = True, 
    # ):

# !!!!!!!!!!!!! IMPORTANT (?) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [4]:
"""
In MeterPremise, can I create a function which will build mp_df_hist given mp_df_curr?
In DOVSOutages I need to build new build_mp_for_outg (and update its use throughout)
    APPARENTLY I already have this, build_active_MP_for_outages or one of other similar functions
    
I need to basically replace everything in DOVSOutages which uses build_mp_for_outg
"""

'\nIn MeterPremise, can I create a function which will build mp_df_hist given mp_df_curr?\nIn DOVSOutages I need to build new build_mp_for_outg (and update its use throughout)\n    APPARENTLY I already have this, build_active_MP_for_outages or one of other similar functions\n    \nI need to basically replace everything in DOVSOutages which uses build_mp_for_outg\n'

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [6]:
#----------------------------------------------------------------------------------------------------
# VARIABLES TO BE SET BY USER!
#----------------------------------------------------------------------------------------------------
save_dfs_to_file   = True
read_dfs_from_file = False
save_end_events    = True

#-------------------------
collect_evs_sum_vw = False

#-------------------------
# run_date is used to collect all results from a given acquisiton run together.
# As such, run_date should be set to the first date of the acquisition run, and
#   SHOULD NOT be changed for each individual date in a run (which typically lasts
#   over the course of days/weeks)
run_date = '20250514'

#-------------------------
date_0   = '2024-07-01' # Lower limit for end events
date_1   = '2024-09-30' # Upper limit for end events

#-------------------------
td_left  = pd.Timedelta('-31D')
td_right = pd.Timedelta('-1D')

#--------------------------------------------------
# NOTE: below, states and opcos should be consistent!
#       i.e., e.g., if states='OH', then opcos should be 'oh' (or None, I suppose)
#-------------------------
# states used to 
#   (1) find transformers which suffered at least one outage from DOVS
#   (2) find all transformers from MeterPremise
# states can be:
#   - a single string, e.g. 'OH'
#   - a list of strings, e.g., ['OH', 'WV']
#   - None
# NOTE: states tend to be upper-case!
states=['OH']

#-------------------------
# opcos used with AMIEndEvents to
#  (1) find the premise numbers which recorded an event between date_0 and date_1.
#  (2) selection/acquisiton of end_device_events
# opcos can be:
#   - a single string, e.g. 'oh'
#   - a list of strings, e.g., ['oh', 'tx']
#   - None
# NOTE: opcos tend to be lower-case!
# NOTE: Acceptable opcos appear to be: ['ap', 'im', 'oh', 'pso', 'swp', 'tx']
opcos='oh'

#-------------------------
# cities = None
cities = ['COLUMBUS']


# TODO!!!!!!!!!!!!!!!!!!
single_zip_xfmrs_only = True

In [7]:
############### NEW
outg_daq = OutageDAQOutg(
    run_date                = run_date, 
    date_0                  = date_0, 
    date_1                  = date_1, 
    collect_evs_sum_vw      = collect_evs_sum_vw,  
    td_left                 = td_left, 
    td_right                = td_right, 
    states                  = states, 
    opcos                   = opcos, 
    cities                  = cities, 
    single_zip_xfmrs_only   = single_zip_xfmrs_only, 
    save_end_events         = save_end_events, 
    save_dfs_to_file        = save_dfs_to_file, 
    read_dfs_from_file      = read_dfs_from_file, 
    base_dir                = os.path.join(
        Utilities.get_local_data_dir(), 
        r'dovs_and_end_events_data'
    ), 
    run_date_subdir_appndx  = None, 
    dates_subdir_appndx     = None, 
)

self.save_dir_base = C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20250514\20240701_20240930\Outages\end_events_method
self.end_events_save_args
	save_to_file : True
	save_dir : C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20250514\20240701_20240930\Outages\end_events_method\EndEvents
	save_name : end_events.csv
	index : True


In [8]:
#--------------------------------------------------
# OUTAGES
#--------------------------------------------------
outg_daq.build_or_load_df_outage_OG(verbose=True)
outg_daq.build_or_load_df_mp_outg(verbose=True)
outg_daq.build_or_load_df_outage_slim()

#--------------------------------------------------
# Build zip code information for transformers, 
# and possibly restrict data to single zipcode transformers only
#--------------------------------------------------
outg_daq.build_or_load_trsf_pole_zips_info(verbose=True)

----------------------------------------------------------------------------------------------------
Finding outages between 2024-07-01 00:00:00 and 2024-09-30 00:00:00 for states=['OH']
--------------------------------------------------

DOVSOutages_SQL: Found opco in kwargs
Converting to OPERATING_UNIT_ID

Input opcos = ['oh']
OPERATING_UNIT_IDs used = [3]
self.sql_outage_full:
SELECT
	DOV.CI_NB,
	DOV.CMI_NB,
	DOV.OUTG_REC_NB,
	DOV.OUTAGE_NB,
	DOV.DT_ON_TS,
	DOV.DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL,
	DOV.STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR,
	DOV.OPERATING_UNIT_ID,
	DOV.STATE_ABBR_TX,
	DOV.MJR_CAUSE_CD,
	DOV.MNR_CAUSE_CD,
	DOV.LOCATION_ID,
	DOV.GIS_CRCT_NB,
	DOV1.OPRTG_UNT_NM,
	DOV3.DVC_TYP_NM,
	DOV3.SHORT_NM AS SHORT_NM_CLR_DEV,
	DOV4.EQUIP_TYP_NM,
	DOV4.SHORT_NM AS SHORT_NM_EQP_TYP,
	DOV5.MJR_CAUSE_NM,
	DOV5.MNR_CAUSE_NM,
	PRIM.OFF_TM,
	PRIM.REST_TM,
	PRIM.PREMISE_NB,
	PRIM.CITY_NM
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
LEFT OUTE

C:\Users\s346557\Documents\Analysis\OutageDAQ.py:535: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_outage_OG = pd.read_sql_query(


self.df_outage_OG.shape = (141505, 26)
# OUTG_REC_NBs     = 1818

time = 42.76395606994629
----------------------------------------------------------------------------------------------------
n_coll     = 102164
batch_size = 10000
n_batches  = 11
10/11
n_coll     = 102164
batch_size = 10000
n_batches  = 11
10/11

            !!!!! WARNING !!!!! In consolidate_df_according_to_fuzzy_overlap_intervals:
            df has values for which df[ovrlp_intrvl_0_col]>=df[ovrlp_intrvl_1_col]!
            Number of violators = 9
            df.shape[0]         = 61902
            Rows containing these violators will be omitted!
        
groups_violating_uniqueness = []
n_coll     = 102125
batch_size = 10000
n_batches  = 11
10/11
trsf_pole_zips_df build time: 2.9486734867095947


In [9]:
#--------------------------------------------------
# Collect Events
#--------------------------------------------------
outg_daq.collect_events(
    batch_size         = None, 
    verbose            = True, 
    n_update           = 1, 
    delete_all_others  = True
)

counter = 0
	wait_time_i=0


BEFORE EXCLUDE PREVIOUSLY RECORDED: n_coll = 17769
n_coll     = 17769
batch_size = 30
n_batches  = 593
1/593
2/593
3/593
4/593
5/593
6/593
7/593
8/593
9/593
10/593
11/593
12/593
13/593
14/593
15/593
16/593
17/593
18/593
19/593
20/593
21/593
22/593
23/593
24/593
25/593
26/593
27/593
28/593
29/593
30/593
31/593
32/593
33/593
34/593
35/593
36/593
37/593
38/593
39/593
40/593
41/593
42/593
43/593
44/593
45/593
46/593
47/593
48/593
49/593
50/593
51/593
52/593
53/593
54/593
55/593
56/593
57/593
58/593
59/593
60/593
61/593
62/593
63/593
64/593
65/593
66/593
67/593
68/593
69/593
70/593
71/593
72/593
73/593
74/593
75/593
76/593
77/593
78/593
79/593
80/593
81/593
82/593
83/593
84/593
85/593
86/593
87/593
88/593
89/593
90/593
91/593
92/593
93/593
94/593
95/593
96/593
97/593
98/593
99/593
100/593
101/593
102/593
103/593
104/593
105/593
106/593
107/593
108/593
109/593
110/593
111/593
112/593
113/593
114/593
115/593
116/593
117/593
118/593
119/593
120/593
121/593
122/593


In [10]:
assert(0)

AssertionError: 

# OLD DEV

### OutageDAQOutg constructor

In [ ]:
cities

In [ ]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
# DFs will be saved in save_dir_base
# Collection of end events files will be saved in os.path.join(save_dir_base, 'EndEvents')
save_dir_base = os.path.join(
    Utilities.get_local_data_dir(), 
    r'dovs_and_end_events_data', 
    run_date, 
    f"{date_0.replace('-','')}_{date_1.replace('-','')}", 
    'Outgs_Full_OLD'
)
#-------------------------
end_events_save_args = dict(
    save_to_file = save_end_events, 
    save_dir     = os.path.join(save_dir_base, 'EndEvents'), 
    save_name    = r'end_events.csv', 
    index        = True
)
#-------------------------
print(f"save_dir_base = {save_dir_base}")
print('end_events_save_args')
for k,v in end_events_save_args.items():
    print(f"\t{k} : {v}")
#-------------------------
if save_dfs_to_file or save_end_events:
    if not os.path.exists(save_dir_base):
        os.makedirs(save_dir_base)
    #-----
    if save_end_events and not os.path.exists(end_events_save_args['save_dir']):
        os.makedirs(end_events_save_args['save_dir'])

In [ ]:
#----------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------
assert(save_dfs_to_file+read_dfs_from_file <=1) # Should never both read and write!
assert(pd.to_datetime(date_1)-pd.to_datetime(date_0) > 2*search_time_window)
#--------------------------------------------------
if not read_dfs_from_file:
    conn_outages = Utilities.get_utldb01p_oracle_connection()
    conn_aws     = Utilities.get_athena_prod_aws_connection()

# ---------------------------------------------------------------
# OUTAGES
# ---------------------------------------------------------------

### build_or_load_df_outage_OG

In [ ]:
#----------------------------------------------------------------------------------------------------
# Find outages between date_0 and date_1 for states
#----------------------------------------------------------------------------------------------------
start=time.time()
print('-----'*20+f'\nFinding outages between {date_0} and {date_1} for states={states}\n'+'-----'*10)
if read_dfs_from_file:
    print(f"Reading df_outage_OG from file: {os.path.join(save_dir_base, 'df_outage_OG.pkl')}")
    df_outage_OG = pd.read_pickle(os.path.join(save_dir_base, 'df_outage_OG.pkl'))
else:
    sql_outage_full = DOVSOutages_SQL.build_sql_std_outage(
        mjr_mnr_cause   = None, 
        include_premise = True, 
        date_range      = [date_0, date_1], 
        states          = states, 
        opcos           = opcos, 
        cities          = cities
    ).get_sql_statement()
    #-----
    print(f'sql_outage_full:\n{sql_outage_full}\n\n')
    #-----
    df_outage_OG = pd.read_sql_query(
        sql_outage_full, 
        conn_outages, 
        dtype = {
            'CI_NB':np.int32, 
            'CMI_NB':np.float64, 
            'OUTG_REC_NB':np.int32
        }
    )
    if save_dfs_to_file:
        df_outage_OG.to_pickle(os.path.join(save_dir_base, 'df_outage_OG.pkl'))
#-----
print(f"df_outage_OG.shape = {df_outage_OG.shape}")
print(f"# OUTG_REC_NBs     = {df_outage_OG['OUTG_REC_NB'].nunique()}")
print(f'\ntime = {time.time()-start}\n'+'-----'*20)

### build_or_load_df_mp_outg

In [ ]:
if read_dfs_from_file:
    # No real reason to read in df_mp_outg_OG, as it's not used after df_mp_outg is built
    # df_mp_outg_OG = pd.read_pickle(os.path.join(save_dir_base, 'df_mp_outg_b4_dupl_rmvl.pkl'))
    df_mp_outg = pd.read_pickle(os.path.join(save_dir_base, 'df_mp_outg_full.pkl'))
else:
    df_mp_outg_OG = OutageDAQOutg.build_active_MP_for_outages_df(
        df_outage            = df_outage_OG, 
        prem_nb_col          = 'PREMISE_NB', 
        is_slim              = False, 
        assert_all_PNs_found = False
    )
    #-----
    df_mp_outg_OG['inst_ts'] = pd.to_datetime(df_mp_outg_OG['inst_ts'])
    df_mp_outg_OG['rmvl_ts'] = pd.to_datetime(df_mp_outg_OG['rmvl_ts'])
    #-----
    if save_dfs_to_file:
        df_mp_outg_OG.to_pickle(os.path.join(save_dir_base, 'df_mp_outg_b4_dupl_rmvl.pkl'))
    #-------------------------
    df_mp_outg = MeterPremise.drop_approx_mp_duplicates(
        mp_df                 = df_mp_outg_OG.copy(), 
        fuzziness             = pd.Timedelta('1 hour'), 
        assert_single_overlap = True, 
        addtnl_groupby_cols   = ['OUTG_REC_NB'], 
        gpby_dropna           = False
    )
    #-----
    if save_dfs_to_file:
        df_mp_outg.to_pickle(os.path.join(save_dir_base, 'df_mp_outg_full.pkl'))

### build_or_load_df_outage_slim

In [ ]:
if read_dfs_from_file:
    df_outage      = pd.read_pickle(os.path.join(save_dir_base, 'df_outage.pkl'))
    df_outage_slim = pd.read_pickle(os.path.join(save_dir_base, 'df_outage_slim.pkl'))
else:
    df_outage = DOVSOutages.merge_df_outage_with_mp(
        df_outage          = df_outage_OG.copy(), 
        df_mp              = df_mp_outg, 
        merge_on_outg      = ['OUTG_REC_NB', 'PREMISE_NB'], 
        merge_on_mp        = ['OUTG_REC_NB', 'prem_nb'], 
        cols_to_include_mp = None, 
        drop_cols          = None, 
        rename_cols        = None, 
        inplace            = True
    )
    #-------------------------
    #df_outage_slim_OLD = DOVSOutages.consolidate_df_outage_OLD(df_outage)
    df_outage_slim     = DOVSOutages.consolidate_df_outage(
        df_outage, 
        addtnl_grpby_cols        = ['trsf_pole_nb'], 
        set_outg_rec_nb_as_index = False
    )

    #-------------------------
    df_outage_slim = DOVSOutages.set_search_time_in_outage_df(
        df_outage          = df_outage_slim, 
        search_time_window = search_time_window
    )
    
    #-------------------------
    if save_dfs_to_file:
        df_outage.to_pickle(os.path.join(save_dir_base, 'df_outage.pkl'))
        df_outage_slim.to_pickle(os.path.join(save_dir_base, 'df_outage_slim.pkl'))    

# Build zip code information for transformers, and possibly restrict data to single zipcode transformers only

### build_or_load_trsf_pole_zips_info

In [ ]:
if read_dfs_from_file:
    mp_for_zips_df    = pd.read_pickle(os.path.join(save_dir_base, 'mp_for_zips_df.pkl'))
    trsf_pole_df_full = pd.read_pickle(os.path.join(save_dir_base, 'trsf_location_info_df.pkl'))
    trsf_pole_zips_df = pd.read_pickle(os.path.join(save_dir_base, 'trsf_pole_zips_df.pkl'))
else:
    zips_dict = OutageDAQ.build_trsf_pole_zips_df(
        field_to_split_and_val = ('premise_nbs', df_mp_outg['prem_nb'].unique().tolist()), 
        states                 = states, 
        opcos                  = opcos, 
        cities                 = cities
    )
    #----------
    trsf_pole_zips_df = zips_dict['trsf_pole_zips_df']
    trsf_pole_df_full = zips_dict['trsf_pole_df_full']
    mp_for_zips_df    = zips_dict['mp_for_zips_df']
    #--------------------------------------------------
    if save_dfs_to_file:
        mp_for_zips_df.to_pickle(   os.path.join(save_dir_base, 'mp_for_zips_df.pkl'))
        trsf_pole_df_full.to_pickle(os.path.join(save_dir_base, 'trsf_location_info_df.pkl'))
        trsf_pole_zips_df.to_pickle(os.path.join(save_dir_base, 'trsf_pole_zips_df.pkl'))
#--------------------------------------------------
if single_zip_xfmrs_only:
    trsf_pole_nzips   = trsf_pole_zips_df.drop(columns=['zip+4']).drop_duplicates()['trsf_pole_nb'].value_counts()
    single_zip_poles  = trsf_pole_nzips[trsf_pole_nzips==1].index.tolist()
    #-----
    trsf_pole_zips_df = trsf_pole_zips_df[trsf_pole_zips_df['trsf_pole_nb'].isin(single_zip_poles)]
    #-----
    df_outage      = df_outage[df_outage['trsf_pole_nb'].isin(single_zip_poles)].copy()
    df_outage_slim = df_outage_slim[df_outage_slim['trsf_pole_nb'].isin(single_zip_poles)].copy()

### collect_events

In [ ]:
print(df_outage_slim['OUTG_REC_NB'].nunique())
print(len(DOVSOutages.get_prem_nbs_from_consolidated_df_outage(df_outage_slim)))

In [ ]:
try:
    del df_outage_OG
    del df_outage
except:
    pass

In [ ]:
#-------------------------
df_construct_type              = DFConstructType.kRunSqlQuery
contstruct_df_args_end_events  = None
addtnl_groupby_cols            = ['OUTG_REC_NB', 'trsf_pole_nb']

cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
# batch_size = 10
batch_size = 30
verbose    = True
n_update   = 1

In [ ]:
end_events_sql_function_kwargs = dict(
    cols_of_interest                  = cols_of_interest_end_dev_event, 
    df_outage                         = df_outage_slim, 
    build_sql_function                = AMIEndEvents_SQL.build_sql_end_events, 
    build_sql_function_kwargs         = dict(
        states = states, 
        opcos  = opcos, 
        cities = cities
    ), 
    join_mp_args                      = False, 
    df_args                           = dict(
        addtnl_groupby_cols = addtnl_groupby_cols, 
        mapping_to_ami      = DfToSqlMap(df_col='PREMISE_NBS', kwarg='premise_nbs', sql_col='aep_premise_nb'), 
        is_df_consolidated  = True
    ), 
    # GenAn - keys_to_pop in GenAn.build_sql_general
    field_to_split                    = 'df_outage', 
    field_to_split_location_in_kwargs = ['df_outage'], 
    save_and_dump                     = True, 
    sort_coll_to_split                = True,
    batch_size                        = batch_size, 
    verbose                           = verbose, 
    n_update                          = n_update
)

In [ ]:
start=time.time()

exit_status = Utilities.run_tryexceptwhile_process(
    func                = AMIEndEvents,
    func_args_dict      = dict(
        df_construct_type         = df_construct_type, 
        contstruct_df_args        = contstruct_df_args_end_events, 
        build_sql_function        = AMIEndEvents_SQL.build_sql_end_events_for_outages, 
        build_sql_function_kwargs = end_events_sql_function_kwargs, 
        init_df_in_constructor    = True, 
        save_args                 = end_events_save_args
    ), 
    max_calls_per_min   = 1, 
    lookback_period_min = 15, 
    max_calls_absolute  = 1000, 
    verbose             = True
)
print(f'exit_status = {exit_status}')

build_time = time.time()-start
print(build_time)

In [ ]:
assert(0)